# Importando bibliotecas

In [1]:
import logging
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd
from stable_baselines3 import DQN
from stable_baselines3.dqn.policies import MlpPolicy
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import time
from datetime import datetime
from torch import nn  # Import the neural network module from PyTorch
import gym
from gym import spaces
from sklearn.model_selection import train_test_split
from stable_baselines3 import DQN
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import sys
import os
sys.path.append(os.path.join('..'))
from classes._exploration import exploration
from classes._Env3WGym import Env3WGym
from classes._LoadInstances import LoadInstances
from classes._Agent import Agent



In [ ]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            try:
                class_code = int(class_path.stem)
            except ValueError:
                # Se não for possível converter para int, pule este diretório
                continue

            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [ ]:
def load_instance_with_numpy(data_path, events_names, columns, well_names):
    data_path = Path(data_path)
  
    real_instances = list(class_and_file_generator(data_path, real=True, simulated=False, drawn=False))

    arrays_list = []  # List to store processed NumPy arrays

    for instance in real_instances:
        class_code, instance_path = instance    
        well, _ = instance_path.stem.split('_')  
             
        if class_code in events_names.keys() and well in well_names:

            # Read the entire CSV file into a NumPy array
            with open(instance_path, 'r') as file:
                header = file.readline().strip().split(',')
                indices = [header.index(col) for col in columns]
                                                
                if 'timestamp' in columns:
                    arr = np.genfromtxt(file, delimiter=',', usecols=indices[1:], dtype=np.float32)
                    arr[np.isinf(arr)] = np.nan
                    timestamp_idx = header.index('timestamp')
                    # Primeiro, vamos carregar todos os timestamps como um array numpy
                    file.seek(0)
                    file.readline()  # Skip the header
                    timestamps = np.genfromtxt(file, delimiter=',', skip_header=0, usecols=timestamp_idx, dtype=str)
                    if isinstance(timestamps, str):
                        timestamps = np.array([timestamps])

                    # Converta os timestamps para o formato desejado antes de aplicar o filtro
                    fmt = "%Y-%m-%d %H:%M:%S.%f"
                    rounded_timestamps = np.array([datetime.strptime(ts, fmt).strftime("%Y-%m-%d %H:%M:%S") for ts in timestamps])

                    # Agora, aplicamos o mesmo filtro usado em 'arr' para 'rounded_timestamps'
                    # Precisamos determinar quais linhas em 'arr' NÃO serão removidas por conter NaN
                    not_nan_rows = ~np.isnan(arr).any(axis=1)

                    # Filtramos 'arr' e 'rounded_timestamps' usando este índice
                    arr_filtered = arr[not_nan_rows]
                    rounded_timestamps_filtered = rounded_timestamps[not_nan_rows]

                    # Agora, ambos 'arr_filtered' e 'rounded_timestamps_filtered' estão alinhados
                    # e podemos concatená-los sem enfrentar o problema de dimensões incompatíveis
                    final_data = np.hstack([rounded_timestamps_filtered.reshape(-1, 1), arr_filtered])

                    arrays_list.append(final_data)
                else:
                    arr = np.genfromtxt(file, delimiter=',', usecols=indices, dtype=np.float32)
                    # Tratando NaN e valores infinitos antes da conversão
                    arr[np.isinf(arr)] = np.nan
                    arr = arr[~np.isnan(arr).any(axis=1)]
                    arr[:, :-1] = arr[:, :-1].astype(np.float32)  # Convert selected columns to float32
                    arr[:, -1] = arr[:, -1].astype(np.int16)  # Convert 'class' column to int16
                    
                    arrays_list.append(arr)               

    # Concatenate all processed NumPy arrays
    final_array = np.concatenate(arrays_list) if arrays_list else np.array([])

    return final_array


In [ ]:
def env3W_dqn(env):

    # Instanciar o modelo DQN com a política MLP
    model = DQN(
        policy=MlpPolicy,
        env=env,
        learning_rate=1e-4,
        buffer_size=10000,
        learning_starts=10000,
        batch_size=32,
        tau=1.0,
        gamma=0.99,
        train_freq=4,
        gradient_steps=1,
        target_update_interval=1000,
        exploration_fraction=0.1,
        exploration_initial_eps=1.0,
        exploration_final_eps=0.01,
        max_grad_norm=10,
        tensorboard_log=None,
        verbose=1,
        device='auto'
    )

    # Treinar o modelo
    model.learn(total_timesteps=int(1.2e5))

    # Salvar o modelo
    model.save('dqn_env3W')

    env.close()

In [ ]:
def env3W_dqn_eval(model, env, n_eval_episodes=1):
    # Inicializa listas para armazenar os dados de cada episódio
    correct_predictions_list = []
    total_predictions_list = []

    for episode in range(n_eval_episodes):
        obs = env.reset()
        done = [False]

        # Reseta contadores para o episódio atual
        correct_predictions = 0
        total_predictions = 0

        while not all(done):
            action, _states = model.predict(obs, deterministic=True)
            obs, rewards, done, info = env.step(action)

            for reward in rewards:
                if reward > 0:
                    correct_predictions += 1
                total_predictions += 1

        # Atualiza a mensagem de log para incluir a recompensa e a porcentagem de acerto
        if total_predictions > 0:  # Evita divisão por zero
            percentage_correct = correct_predictions / total_predictions
            #logging.info(f"[{episode}] Recompensa: {correct_predictions}, Porcentagem de acerto: {percentage_correct:.2f}.")
            #print(f"[{episode}] Recompensa: {correct_predictions}, Porcentagem de acerto: {percentage_correct:.2f}.")
        else:
            logging.info(f"Sem predições.")
            #print(f"[{episode}] Sem predições.")

        # Armazena os dados do episódio atual nas listas
        correct_predictions_list.append(correct_predictions)
        total_predictions_list.append(total_predictions)

    # Calcula a precisão geral
    accuracy = sum(correct_predictions_list) / sum(total_predictions_list) if sum(total_predictions_list) > 0 else 0

    
    return accuracy

In [ ]:
if __name__ == '__main__':

    events_names = {
        0: 'Normal',
        # 1: 'Abrupt Increase of BSW',
        # 2: 'Spurious Closure of DHSV',
        # 3: 'Severe Slugging',
        4: 'Flow Instability',
        # 5: 'Rapid Productivity Loss',
        # 6: 'Quick Restriction in PCK',
        # 7: 'Scaling in PCK',
        # 8: 'Hydrate in Production Line'
    }

    event_name = [value for key, value in events_names.items() if key != 0]


    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_filename = f'/home/Reinforcement-Learning/logs/{current_time}_{event_name}-log.txt'
    # Configuração do Logging
    logging.basicConfig(filename=log_filename, filemode='w', level=logging.INFO, format='[%(levelname)s]\t%(asctime)s - %(message)s', datefmt='%d/%m/%Y %I:%M:%S %p', force=True)

    path_dataset = '/home/dataset'    

    columns = [       
        'P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        #'P-JUS-CKGL',
        #'T-JUS-CKGL',
        #'QGL',
        'class'
    ]
   
    well_names = [f'WELL-{i:05d}' for i in range(1, 19)]
    
    logging.info(f'Iniciando carregamento do dataset')
    dataset = load_instance_with_numpy(data_path = path_dataset, events_names = events_names, columns = columns, well_names = well_names)    
    logging.info(f'Fim carregamento do dataset')
    
    logging.info(f'Iniciando divisão do dataset em treino e teste')
        
    # Definindo a porcentagem para divisão entre treino e teste
    train_percentage = 0.8  # 80% para treino

    # Inicializando listas para guardar índices de treino e teste
    train_indices = []
    test_indices = []

    # Processamento genérico para cada classe
    for event in np.unique(dataset[:, -1]):
        # Selecionando índices para a classe atual        
        class_indices = np.where(dataset[:, -1] == event)[0]
        
        # Logando o número de amostras por classe
        print(f'Número de amostras da classe {event}: {len(class_indices)}')
        logging.info(f'Número de amostras da classe {event}: {len(class_indices)}')
        
        # Dividindo os índices da classe atual em treino e teste
        class_train_indices, class_test_indices = train_test_split(class_indices, train_size=train_percentage, random_state=42)
        
        # Logando o número de amostras de treino e teste
        logging.info(f'Número de amostras de treino da classe {event}: {len(class_train_indices)}')
        logging.info(f'Número de amostras de teste da classe {event}: {len(class_test_indices)}')
        
        # Adicionando aos índices gerais de treino e teste
        train_indices.extend(class_train_indices)
        test_indices.extend(class_test_indices)

    # Convertendo listas para arrays numpy para futura manipulação
    train_indices = np.array(train_indices)
    test_indices = np.array(test_indices)

    # Embaralhando os índices (opcional, dependendo da necessidade)
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)

    # Criando conjuntos de dados de treino e teste
    dataset_train = dataset[train_indices]
    dataset_test = dataset[test_indices]

    # Dividindo em features (X) e target (y)
    X_train, y_train = dataset_train[:, :-1], dataset_train[:, -1]
    X_test, y_test = dataset_test[:, :-1], dataset_test[:, -1]

    # Escalonando as features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    

    # Se necessário, você pode combinar as features escalonadas e o target para formar os datasets finais
    dataset_train_scaled = np.column_stack((X_train_scaled, y_train))
    dataset_test_scaled = np.column_stack((X_test_scaled, y_test))
    logging.info(f'Fim divisão do dataset em treino e teste')
    
    logging.info(f'Iniciando treinamento do modelo DQN')
    start_time = time.time()
    np.random.seed(42)  # Para reprodutibilidade
    shuffled_indices = np.random.permutation(len(dataset_train_scaled))
    dataset_shuffled = dataset_train_scaled[shuffled_indices]

    # Dividir o dataset embaralhado para os ambientes
    n_envs = 5
    split_datasets = np.array_split(dataset_shuffled, n_envs)

    # Função modificada para criar ambientes com partes específicas do dataset
    def create_env(dataset_part):
        return Env3WGym(dataset_part)
    # Crie o ambiente (modifique conforme necessário para seu caso de uso)
    #env = make_vec_env(lambda: Env3WGym(dataset_train_scaled), n_envs=5)
    envs = make_vec_env(lambda: create_env(split_datasets.pop(0)), n_envs=n_envs)
    env3W_dqn(envs)  # Chamar a função para treinar o modelo
    print(f"Tempo de Treinamento DQN: {time.time() - start_time}")
    logging.info(f"Tempo de Treinamento DQN: {time.time() - start_time}")
    logging.info(f'Fim treinamento do modelo DQN')

    logging.info(f'Iniciando avaliação do modelo de teste DQN')
    dqn_model = DQN.load('dqn_env3W')
    np.random.seed(42)  # Para reprodutibilidade
    shuffled_indices = np.random.permutation(len(dataset_test_scaled))
    dataset_shuffled = dataset_test_scaled[shuffled_indices]

    # Dividir o dataset embaralhado para os ambientes
    n_envs = 5
    split_datasets = np.array_split(dataset_shuffled, n_envs)

    # Função modificada para criar ambientes com partes específicas do dataset
    def create_env(dataset_part):
        return Env3WGym(dataset_part)
    
    envs = make_vec_env(lambda: create_env(split_datasets.pop(0)), n_envs=n_envs)
    #env = make_vec_env(lambda: Env3WGym(dataset_test_scaled), n_envs=5)
    # Executar a função de avaliação
    accuracy = env3W_dqn_eval(dqn_model, envs)
    print(f'Acurácia: {accuracy}')
    logging.info(f'Acurácia: {accuracy}')
    logging.info(f'Fim avaliação do modelo de teste DQN')

    logging.info(f'Iniciando o teste do modelo de teste DQN')
    columns = [
        'timestamp',
        'P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        #'P-JUS-CKGL',
        #'T-JUS-CKGL',
        #'QGL',
        'class'       
    ]
    
    logging.info(f'Iniciando carregamento do dataset de teste')
    dataset_test = load_instance_with_numpy(data_path = '/home/dataset_test', events_names = events_names, columns = columns, well_names = well_names)
    # Escalone as features dataset_test
    logging.info(f'Fim carregamento do dataset de teste')

    logging.info(f'Iniciando escalonamento do dataset de teste')
    X_test = dataset_test[:, 1:-1]
    X_test_scaled = scaler.transform(X_test)
    dataset_test = np.column_stack((dataset_test[:, 0], X_test_scaled, dataset_test[:, -1]))
    logging.info(f'Fim escalonamento do dataset de teste')

    logging.info(f'Iniciando predição de uma instância para teste DQN')
    array_action_pred = []
    for i in range(0, len(dataset_test)):
        obs = dataset_test[i, 1:-1].astype(np.float32)
        action, _states = dqn_model.predict(obs, deterministic=True)  
        array_action_pred.append(action)

    
    expanded_array = np.column_stack((dataset_test, array_action_pred))
    logging.info(f'Fim predição da instância de teste DQN')    
   
    
    df = pd.DataFrame(expanded_array, columns = ['timestamp', 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'class', 'action'])
    df.set_index('timestamp', inplace=True)
    df[['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']] = df[['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']].astype('float32')
    df['class'] = df['class'].astype(float).astype('int16')
    df['action'] = df['action'].astype(float).astype('int16')


    explora = exploration(df)
    explora.plot_sensor(sensor_columns = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP'], _title = '{event_name} - DQN')
    logging.info(f'Concluído a execução do aprendizado por reforço')

In [ ]:
if __name__ == '__main__':

    events_names = {
        0: 'Normal',
        # 1: 'Abrupt Increase of BSW',
        # 2: 'Spurious Closure of DHSV',
        # 3: 'Severe Slugging',
        4: 'Flow Instability',
        # 5: 'Rapid Productivity Loss',
        # 6: 'Quick Restriction in PCK',
        # 7: 'Scaling in PCK',
        # 8: 'Hydrate in Production Line'
    }

    event_name = [value for key, value in events_names.items() if key != 0]
    

    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_filename = f'..\\..\\logs\\{current_time}_{event_name}-log.txt'
    # Configuração do Logging
    logging.basicConfig(filename=log_filename, filemode='w', level=logging.INFO, format='[%(levelname)s]\t%(asctime)s - %(message)s', datefmt='%d/%m/%Y %I:%M:%S %p', force=True)

    path_dataset = '..\\..\\..\\dataset'     

    columns = [ 
        'timestamp',      
        'P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        #'P-JUS-CKGL',
        #'T-JUS-CKGL',
        #'QGL',
        'class'
    ]
   
    well_names = [f'WELL-{i:05d}' for i in range(1, 19)]

    instances = LoadInstances(path_dataset)
    
    logging.info(f'Iniciando carregamento do dataset')
    dataset = instances.load_instance_with_numpy(events_names, columns)    
    logging.info(f'Fim carregamento do dataset')
    
    logging.info(f'Iniciando divisão do dataset em treino e teste')
        
    # Definindo a porcentagem para divisão entre treino e teste
    train_percentage = 0.8  # 80% para treino

    # Inicializando listas para guardar índices de treino e teste
    train_indices = []
    test_indices = []

    # Processamento genérico para cada classe
    for event in np.unique(dataset[:, -1]):
        # Selecionando índices para a classe atual        
        class_indices = np.where(dataset[:, -1] == event)[0]
        
        # Logando o número de amostras por classe
        print(f'Número de amostras da classe {event}: {len(class_indices)}')
        logging.info(f'Número de amostras da classe {event}: {len(class_indices)}')
        
        # Dividindo os índices da classe atual em treino e teste
        class_train_indices, class_test_indices = train_test_split(class_indices, train_size=train_percentage, random_state=42)
        
        # Logando o número de amostras de treino e teste
        logging.info(f'Número de amostras de treino da classe {event}: {len(class_train_indices)}')
        logging.info(f'Número de amostras de teste da classe {event}: {len(class_test_indices)}')
        
        # Adicionando aos índices gerais de treino e teste
        train_indices.extend(class_train_indices)
        test_indices.extend(class_test_indices)

    # Convertendo listas para arrays numpy para futura manipulação
    train_indices = np.array(train_indices)    
    test_temp_indices = np.array(test_indices)       

    test_indices, validation_indices = train_test_split(test_temp_indices, test_size=0.5, random_state=42)

    # Embaralhando os índices (opcional, dependendo da necessidade)
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)
    

    # Criando conjuntos de dados de treino e teste
    dataset_train = dataset[train_indices]
    dataset_test = dataset[test_indices]
    dataset_validation = dataset[validation_indices]
    

    # Dividindo em features (X) e target (y)
    X_train, y_train = dataset_train[:, :-1], dataset_train[:, -1]
    X_test, y_test = dataset_test[:, :-1], dataset_test[:, -1]
    X_validation, y_validation = dataset_validation[:, :-1], dataset_validation[:, -1]

    # Delete a primeira coluna (timestamp) das features
    X_train = np.delete(X_train, 0, axis=1)
    X_test = np.delete(X_test, 0, axis=1)

    # Escalonando as features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_validation_scaled = scaler.transform(X_validation[:, 1:])   
   

    # Se necessário, você pode combinar as features escalonadas e o target para formar os datasets finais
    dataset_train_scaled = np.column_stack((X_train_scaled, y_train))
    dataset_test_scaled = np.column_stack((X_test_scaled, y_test))
    dataset_validation_scaled = np.column_stack((X_validation_scaled, y_validation))
    logging.info(f'Fim divisão do dataset em treino e teste')
    
    logging.info(f'Iniciando treinamento do modelo DQN')
    
    start_time = time.time()
    agente = Agent('..\\models\\dqn_env3W')

    agente.env3W_dqn(dataset_train_scaled, n_envs = 5)  
    print(f"Tempo de Treinamento DQN: {time.time() - start_time}")
    logging.info(f"Tempo de Treinamento DQN: {time.time() - start_time}")

    logging.info(f'Fim treinamento do modelo DQN')

    logging.info(f'Iniciando avaliação do modelo de teste DQN')    
    
    accuracy, dqn_model = agente.env3W_dqn_eval(dataset_test_scaled, n_envs = 5)
    print(f'Acurácia: {accuracy}')
    logging.info(f'Acurácia: {accuracy}')
    logging.info(f'Fim avaliação do modelo de teste DQN')

    # Obtendo os índices que ordenariam a primeira coluna
    sort_indices = np.argsort(dataset_validation_scaled[:, 0])

    # Usando esses índices para reordenar todo o array
    dataset_validation_sorted = dataset_validation_scaled[sort_indices]

    # Inicializando previous_timestamp como None para a primeira comparação
    previous_datetime = None

    for row in dataset_validation_sorted:
        current_datetime = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
        
        if previous_datetime and (current_datetime - previous_datetime).total_seconds() / 3600 > 1:
            # Imprime a hora do timestamp atual se a diferença for maior que 1 hora
            print(current_datetime.hour)
        
        previous_datetime = current_datetime
        
    # Inicializando a lista para armazenar os sub-datasets
    datasets = []
    current_dataset = []

    # Inicializando previous_datetime como None para a primeira comparação
    previous_datetime = None

    for row in dataset_validation_sorted:
        current_datetime = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
        
        # Verifica se é a primeira iteração ou se a diferença é maior que 1 hora
        if previous_datetime is None or (current_datetime - previous_datetime).total_seconds() / 3600 > 1:
            # Se não for a primeira iteração e a condição for verdadeira, inicia um novo dataset
            if current_dataset:
                datasets.append(np.array(current_dataset))
                current_dataset = []
        
        # Adiciona o registro atual ao dataset corrente
        current_dataset.append(row)
        previous_datetime = current_datetime

    # Não esqueça de adicionar o último dataset após terminar o loop
    if current_dataset:
        datasets.append(np.array(current_dataset))
    
    count = -1

    for dataset_test in datasets:
        count += 1
        logging.info(f'Iniciando predição de uma instância para teste DQN')
        array_action_pred = []
        for i in range(0, len(dataset_test)):
            obs = dataset_test[i, 1:-1].astype(np.float32)
            action, _states = dqn_model.predict(obs, deterministic=True)  
            array_action_pred.append(action)

        
        expanded_array = np.column_stack((dataset_test, array_action_pred))
        logging.info(f'Fim predição da instância de teste DQN')    
    
        
        df = pd.DataFrame(expanded_array, columns = ['timestamp', 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'class', 'action'])
        df.set_index('timestamp', inplace=True)
        df[['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']] = df[['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']].astype('float32')
        df['class'] = df['class'].astype(float).astype('int16')
        df['action'] = df['action'].astype(float).astype('int16')


        explora = exploration(df)
        explora.plot_sensor(sensor_columns = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP'], _title = '[{count}] - {event_name} - DQN')
        logging.info(f'Concluído a execução do aprendizado por reforço')

In [2]:
events_names = {
    0: 'Normal',
    # 1: 'Abrupt Increase of BSW',
    # 2: 'Spurious Closure of DHSV',
    # 3: 'Severe Slugging',
    4: 'Flow Instability',
    # 5: 'Rapid Productivity Loss',
    # 6: 'Quick Restriction in PCK',
    # 7: 'Scaling in PCK',
    # 8: 'Hydrate in Production Line'
}

event_name = [value for key, value in events_names.items() if key != 0]

path_dataset = '..\\..\\..\\dataset'    

columns = [ 
    'timestamp',      
    'P-PDG',
    'P-TPT',
    'T-TPT',
    'P-MON-CKP',
    'T-JUS-CKP',
    #'P-JUS-CKGL',
    #'T-JUS-CKGL',
    #'QGL',
    'class'
]

well_names = [f'WELL-{i:05d}' for i in range(1, 19)]

instances = LoadInstances(path_dataset)


dataset = instances.load_instance_with_numpy(events_names, columns)    
    
# Definindo a porcentagem para divisão entre treino e teste
train_percentage = 0.8  # 80% para treino

# Inicializando listas para guardar índices de treino e teste
train_indices = []
test_indices = []

# Processamento genérico para cada classe
for event in np.unique(dataset[:, -1]):
    # Selecionando índices para a classe atual        
    class_indices = np.where(dataset[:, -1] == event)[0]
    
    # Logando o número de amostras por classe
    print(f'Número de amostras da classe {event}: {len(class_indices)}')
    
    
    # Dividindo os índices da classe atual em treino e teste
    class_train_indices, class_test_indices = train_test_split(class_indices, train_size=train_percentage, random_state=42)
    
    # Logando o número de amostras de treino e teste
    logging.info(f'Número de amostras de treino da classe {event}: {len(class_train_indices)}')
    logging.info(f'Número de amostras de teste da classe {event}: {len(class_test_indices)}')
    
    # Adicionando aos índices gerais de treino e teste
    train_indices.extend(class_train_indices)
    test_indices.extend(class_test_indices)

# Convertendo listas para arrays numpy para futura manipulação
train_indices = np.array(train_indices)    
test_temp_indices = np.array(test_indices)       

test_indices, validation_indices = train_test_split(test_temp_indices, test_size=0.5, random_state=42)

# Embaralhando os índices (opcional, dependendo da necessidade)
np.random.shuffle(train_indices)
np.random.shuffle(test_indices)


# Criando conjuntos de dados de treino e teste
dataset_train = dataset[train_indices]
dataset_test = dataset[test_indices]
dataset_validation = dataset[validation_indices]


# Dividindo em features (X) e target (y)
X_train, y_train = dataset_train[:, :-1], dataset_train[:, -1]
X_test, y_test = dataset_test[:, :-1], dataset_test[:, -1]
X_validation, y_validation = dataset_validation[:, :-1], dataset_validation[:, -1]

# Delete a primeira coluna (timestamp) das features
X_train = np.delete(X_train, 0, axis=1)
X_test = np.delete(X_test, 0, axis=1)

# Escalonando as features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_validation_scaled = np.column_stack((X_validation[:, 0], scaler.transform(X_validation[:, 1:])))   


# Se necessário, você pode combinar as features escalonadas e o target para formar os datasets finais
dataset_train_scaled = np.column_stack((X_train_scaled, y_train))
dataset_test_scaled = np.column_stack((X_test_scaled, y_test))
dataset_validation_scaled = np.column_stack((X_validation_scaled, y_validation))

# Obtendo os índices que ordenariam a primeira coluna
sort_indices = np.argsort(dataset_validation_scaled[:, 0])

# Usando esses índices para reordenar todo o array
dataset_validation_sorted = dataset_validation_scaled[sort_indices]

# Inicializando previous_timestamp como None para a primeira comparação
previous_datetime = None

for row in dataset_validation_sorted:
    current_datetime = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
    
    if previous_datetime and (current_datetime - previous_datetime).total_seconds() / 3600 > 1:
        # Imprime a hora do timestamp atual se a diferença for maior que 1 hora
        print(current_datetime.hour)
    
    previous_datetime = current_datetime
    
# Inicializando a lista para armazenar os sub-datasets
datasets = []
current_dataset = []

# Inicializando previous_datetime como None para a primeira comparação
previous_datetime = None

for row in dataset_validation_sorted:
    current_datetime = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
    
    # Verifica se é a primeira iteração ou se a diferença é maior que 1 hora
    if previous_datetime is None or (current_datetime - previous_datetime).total_seconds() / 3600 > 1:
        # Se não for a primeira iteração e a condição for verdadeira, inicia um novo dataset
        if current_dataset:
            datasets.append(np.array(current_dataset))
            current_dataset = []
    
    # Adiciona o registro atual ao dataset corrente
    current_dataset.append(row)
    previous_datetime = current_datetime

# Não esqueça de adicionar o último dataset após terminar o loop
if current_dataset:
    datasets.append(np.array(current_dataset))
    
    

Número de amostras da classe 0.0: 8430978
Número de amostras da classe 4.0: 2460270
23
16
1
15
16
0
6
20
11
2
22
2
0
20
12
18
19
6
10
18
21
5
21
19
0
22
18
21
5
6
3
9
18
16
17
8
18
